# Components

In [2]:
import getpass
import os
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_chroma import Chroma

gpt4all_embd = GPT4AllEmbeddings()

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=gpt4all_embd,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [3]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("LLAMA_API_KEY"),
    base_url="https://api.llama-api.com/"
)

def ask_gpt(prompt) -> str:
    response = client.chat.completions.create(
        model="llama3.3-70b",
        messages=[
            # {
            #     "role": "system", 
            #     "content": "Always respond with 'Yes' or 'No' when answering questions."
            # },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        # max_tokens=512,
        # temperature=0,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        # n = 1,
        stream=False
    )
    return response.choices[0].message.content

In [4]:
template = "" \
"You are an assistant specialized in question-answering surgical tasks." \
"Use the retrieved context below to provide a concise, clinically accurate answer to the question." \
"Clearly reference relevant guidelines or sources from the context where possible." \
"If you cannot answer based on the provided information, say you don't know." \
"Keep the answer clear and concise, but provide enough detail to be medically useful." \
"Question: {question}" \
"Context: {context}" \
"Answer:"

task1 = "Checking patient records for missing clinical investigation data."

task2 = "Identifying and flagging investigation results outside of normal ranges."

task3 = "Developing recommendations for next management steps based on national surgical guidelines."

task4 = "Preparing structured operative notes based upon recommended management steps."

In [5]:
from langchain import hub
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

prompt = ""

# Define state for application
class State(TypedDict):
    sauce: str
    question: str
    context: List[Document]
    answer: str

def indexing(state: State):
    # Load and chunk contents of the blog
    # loader = OnlinePDFLoader(state["sauce"])
    # docs = loader.load()

    from langchain.document_loaders import PyMuPDFLoader
    loader = PyMuPDFLoader(state["sauce"])
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    all_splits = text_splitter.split_documents(docs)

    # Index chunks
    _ = vector_store.add_documents(documents=all_splits)

    # Define prompt for question-answering
    prompt = template


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke(
        {"question": state["question"], "context": docs_content}
    ).to_messages()
    response = ask_gpt(messages[0].content)
    return {"answer": response}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([indexing, retrieve, generate])
graph_builder.add_edge(START, "indexing")
graph = graph_builder.compile()

In [6]:
response = graph.invoke(
    {"sauce": "https://www.ahajournals.org/doi/pdf/10.1161/CIR.0000000000001038"},
    {"question": task1}
    )
print(response["answer"])

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


ValueError: Check the url of your file; returned status code 403

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
